# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Question 1

In [2]:
! pip install beautifulsoup4
! pip install lxml
! pip install html5lib
! pip install requests

import pandas as pd
import html
from bs4 import BeautifulSoup
import requests
import urllib.request


In [3]:
!wget -q -O 'Toronto_data.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [146]:
with open('Toronto_data.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [147]:
#table = soup.find('table',class_='wikitable sortable')
table = soup.find('table')
#sum = table.td.text
#print(table)

In [148]:
column_names = ['Postcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [149]:
for tr in table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [150]:
df=df[df.Borough != 'Not assigned']
#df=df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [151]:
#df[df.Neighbourhood =='Not assigned']=df.Borough
df[df['Neighbourhood']=='Not assigned']=df['Borough']

In [152]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [153]:
temp_df=df.groupby('Postcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))

In [154]:
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [155]:
df_merge = pd.merge(df, temp_df, on='Postcode')

In [156]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [157]:
df_merge.drop_duplicates(inplace=True)

In [158]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [159]:
df_merge.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park


In [160]:
df_merge.shape

(103, 3)

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Question 2

In [141]:
!wget -q -O 'geo_data.csv' http://cocl.us/Geospatial_data

In [161]:
df_can = pd.read_csv('geo_data.csv')

In [162]:
df_can.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [168]:
df_geo = df_can.rename(columns={"Postal Code": "Postcode"})
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [169]:
df_joined = pd.merge(df, df_geo, on='Postcode')

In [170]:
# Merged Dataframe 
df_joined.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto - Question 3

In [172]:
# Getting co ordinates of Toronto
from geopy.geocoders import Nominatim
address = 'Toronto City, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.7189883, -79.44157.


In [177]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be 

In [195]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_joined['Latitude'], df_joined['Longitude'], df_joined['Borough'], df_joined['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Let's simplify the above map and segment and cluster only Borough that has the word Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [196]:
Toronto_data = df_joined[df_joined['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


In [198]:
# create map of Borough which has word only Toronto using latitude and longitude values
map_borough = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_borough)  
    
map_borough

In [199]:
CLIENT_ID = 'NYUVVZZHTBPX0UOTHNXP1ZMB4WCN5VIL53QLZOLYYYIUVRXM' # your Foursquare ID
CLIENT_SECRET = 'BRK0VQ2BZOEJZHMBCC44OWGJEYT4WNAC13QJIJNFHD0YIZ1B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NYUVVZZHTBPX0UOTHNXP1ZMB4WCN5VIL53QLZOLYYYIUVRXM
CLIENT_SECRET:BRK0VQ2BZOEJZHMBCC44OWGJEYT4WNAC13QJIJNFHD0YIZ1B


#### Let's explore the first neighborhood in our dataframe.

In [200]:
Toronto_data.loc[0, 'Neighbourhood']

'Harbourfront'

Get the neighborhood's latitude and longitude values.

In [203]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Harbourfront within a radius of 500 meters.

In [204]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NYUVVZZHTBPX0UOTHNXP1ZMB4WCN5VIL53QLZOLYYYIUVRXM&client_secret=BRK0VQ2BZOEJZHMBCC44OWGJEYT4WNAC13QJIJNFHD0YIZ1B&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [205]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3d57a3bf7dde002c027188'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [206]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [208]:
from pandas.io.json import json_normalize 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [209]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

51 venues were returned by Foursquare.


In [210]:
print('The top 51 venues are: ', nearby_venues['name'])

The top 51 venues are:  0                                    Roselle Desserts
1                                       Tandem Coffee
2     Toronto Cooper Koo Family Cherry St YMCA Centre
3                                 Body Blitz Spa East
4                                  Morning Glory Cafe
5                                      Impact Kitchen
6                              Figs Breakfast & Lunch
7                            Dominion Pub and Kitchen
8                                     Corktown Common
9                                  The Extension Room
10                   The Distillery Historic District
11                                SOMA chocolatemaker
12                           Distillery Sunday Market
13                                    Sumach Espresso
14                                     Rooster Coffee
15                                               Arvo
16                                           Cacao 70
17                                          Starbucks
18  